In [1]:
import os
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")  # Report only TF errors by default
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from fredapi import Fred
from dataset import Dataset, BitcoinDataset, EthereumDataset, LitecoinDataset



In [2]:
#controling whether tensorflow does recognize my GPU
tf.config.get_visible_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
btc_dataset = BitcoinDataset()
eth_dataset = EthereumDataset()
ltc_dataset = LitecoinDataset()

In [69]:

fred = Fred(api_key="ee915eacae9f30debeafbd04ea173709")
macro_data = fred.get_series("CPIAUCSL")

In [70]:
macro_data

1947-01-01     21.480
1947-02-01     21.620
1947-03-01     22.000
1947-04-01     22.000
1947-05-01     21.950
               ...   
2023-06-01    303.841
2023-07-01    304.348
2023-08-01    306.269
2023-09-01    307.481
2023-10-01    307.619
Length: 922, dtype: float64

In [71]:
macro_data.resample("D").ffill()

1947-01-01     21.480
1947-01-02     21.480
1947-01-03     21.480
1947-01-04     21.480
1947-01-05     21.480
               ...   
2023-09-27    307.481
2023-09-28    307.481
2023-09-29    307.481
2023-09-30    307.481
2023-10-01    307.619
Freq: D, Length: 28033, dtype: float64

In [60]:
BTC_Ticker = yf.Ticker("BTC-USD")
BTC_Data = BTC_Ticker.history(period="max")
ETH_Ticker = yf.Ticker("ETH-USD")
ETH_Data = ETH_Ticker.history(period="max")
SPY_Ticker = yf.Ticker("SPY")
SPY_Data = SPY_Ticker.history(period="max") 
DJI_Ticker = yf.Ticker("^DJI")
DJI_Data = DJI_Ticker.history(period="max")
GOLD_futures_Ticker = yf.Ticker("GC=F")
GOLD_futures_Data = GOLD_futures_Ticker.history(period="max")